In [3]:
# default_exp data.ts_loader_pinche

In [4]:
#hide
%load_ext autoreload
%autoreload 2

In [5]:
#export
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
from nixtla.data.ts_dataset import TimeSeriesDataset
from collections import defaultdict

In [40]:
#export
class TimeSeriesLoader(object):
    def __init__(self,
                 ts_dataset:TimeSeriesDataset,
                 model:str,
                 offset:int,
                 window_sampling_limit: int, 
                 input_size: int,
                 output_size: int,
                 idx_to_sample_freq: int, #TODO: not active yet
                 batch_size: int,
                 train_loader: bool):
        """
        """
        self.ts_dataset = ts_dataset # Pass by reference
        self.model = model
        self.offset = offset
        self.window_sampling_limit = window_sampling_limit
        self.input_size = input_size
        self.output_size = output_size
        self.idx_to_sample_freq = idx_to_sample_freq
        self.batch_size = batch_size
        self.train_loader = train_loader
        self._is_train = True

        assert offset < self.ts_dataset.max_len, 'Offset must be smaller than max_len'

        self.window_sampling_idx = self._update_windows_sampling_idx()
        
        #TODO: cambiar estos prints
        # print('X: time series features, of shape (#series,#times,#features): \t' + str(X.shape))
        # print('Y: target series (in X), of shape (#series,#times): \t \t' + str(Y.shape))
        # print('S: static features, of shape (#series,#features): \t \t' + str(S.shape))

    def _update_windows_sampling_idx(self):
        # Filter sampling_mask with offset and window_sampling_limit
        last_ds = self.ts_dataset.max_len - self.offset
        first_ds = max(last_ds - self.window_sampling_limit, 0)
        filtered_outsample_mask = self.ts_dataset.ts_tensor[:, self.ts_dataset.t_cols.index('outsample_mask'), first_ds:last_ds]
        filtered_ts_train_mask = self.ts_dataset.ts_train_mask[first_ds:last_ds]

        # Get indices of train/validation windows
        if self.train_loader:
            train_mask =  filtered_outsample_mask * filtered_ts_train_mask
            indices = np.argwhere(train_mask > 0)
        else:
            val_mask = filtered_outsample_mask * (1-filtered_ts_train_mask)
            indices = np.argwhere(val_mask > 0)

        #To change relative position of filtered tensor to global position
        indices[:, 1] += first_ds

        #Loop for each serie to extract window_sampling_idx
        window_sampling_idx = []
        for i in range(self.ts_dataset.n_series):
            ts_idx = indices[indices[:, 0] == i]
            window_sampling_idx.append(list(ts_idx[:, 1]))
        return window_sampling_idx

    def __iter__(self):
        while True:
            if self._is_train:
                sampled_ts_indices = np.random.randint(self.ts_dataset.n_series, size=self.batch_size)
            else:
                sampled_ts_indices = range(self.ts_dataset.n_series)

            batch_dict = defaultdict(list)
            for index in sampled_ts_indices:
                batch_i = self.__get_item__(index)
                for key in batch_i:
                    batch_dict[key].append(batch_i[key])

            batch = defaultdict(list)
            for key in batch_dict:
                batch[key] = np.stack(batch_dict[key])

            yield batch

    def __get_item__(self, index):
        if self.model == 'nbeats':
            return self._nbeats_batch(index)
        elif self.model == 'esrnn':
            assert 1<0, 'hacer esrnn'
        else:
            assert 1<0, 'error'

    def _nbeats_batch(self, index):
        insample = np.zeros((self.ts_dataset.n_channels, self.input_size), dtype=float)
        insample_mask = np.zeros(self.input_size)
        outsample = np.zeros((self.ts_dataset.n_channels, self.output_size), dtype=float)
        outsample_mask = np.zeros(self.output_size)
            
        ts = self.ts_dataset.ts_tensor[index]
        len_ts = self.ts_dataset.len_series[index]
        init_ts = max(self.ts_dataset.max_len-len_ts, self.ts_dataset.max_len-self.offset-self.window_sampling_limit) #TODO: precomputar en loader

        assert self.ts_dataset.max_len-self.offset > init_ts, f'Offset too big for serie {index}'
        if self._is_train:
            cut_point = np.random.choice(self.window_sampling_idx[index],1)[0] # Sampling from available cuts for ts "index"
        else:
            cut_point = max(self.ts_dataset.max_len-self.offset, self.input_size)
        
        insample_window = ts[:-2, max(0, cut_point - self.input_size):cut_point] # se saca mask channel del final
        insample_mask_start = min(self.input_size, cut_point - init_ts) #In case cut_point is close to init_ts, because series are padded
        # print('ts', ts)
        # print('insample', insample)
        # print('insample_window', insample_window)
        # print('self.window_sampling_idx[index]',self.window_sampling_idx[index])
        # print('cut_point', cut_point)
        # print('----')
        insample[:, -insample_window.shape[1]:] = insample_window
        insample_mask[-insample_mask_start:] = 1.0

        if self._is_train:
            #se saca mask channel del final
            outsample_window = ts[:-2, cut_point:min(self.ts_dataset.max_len - self.offset, cut_point + self.output_size)]
        else:
            #se saca mask channel del final
            outsample_window = ts[:-2, cut_point:min(self.ts_dataset.max_len, cut_point + self.output_size)]

        # First mask is to filter after offset, second mask to filter ts validation
        outsample[:, :outsample_window.shape[1]] = outsample_window 
        outsample_mask[:outsample_window.shape[1]] = 1.0
        outsample_mask[:outsample_window.shape[1]] = outsample_mask[:outsample_window.shape[1]] * \
                                                     self.ts_dataset.ts_train_mask[cut_point:(cut_point+outsample_window.shape[1])]

        insample_y = insample[0, :]
        insample_x_t = insample[1:, :]

        outsample_y = outsample[0, :]
        outsample_x_t = outsample[1:, :]

        x_s = self.ts_dataset.x_s[index, :]

        sample = {'insample_y':insample_y, 'insample_x_t':insample_x_t, 'insample_mask':insample_mask,
                  'outsample_y':outsample_y, 'outsample_x_t':outsample_x_t, 'outsample_mask':outsample_mask,
                  'x_s':x_s}

        return sample

    def update_offset(self, offset):
        if offset == self.offset:
            return # Avoid extra computation
        self.offset = offset

    def get_meta_data_var(self, var):
        """
        """
        return self.ts_dataset.get_meta_data_var(var)

    def get_n_variables(self):
        return self.ts_dataset.n_x_t, self.ts_dataset.n_s_t

    def get_n_series(self):
        return self.ts_dataset.n_series

    def get_max_len(self):
        return self.ts_dataset.max_len

    def get_n_channels(self):
        return self.ts_dataset.n_channels

    def get_frequency(self):
        return self.ts_dataset.frequency

    def train(self):
        self._is_train = True

    def eval(self):
        self._is_train = False

In [41]:
from nixtla.data.datasets.tourism import Tourism, TourismInfo
tourism_dataset = Tourism.load(directory='data', group=TourismInfo.groups[0])
tourism_dataset.ts_tensor

Processing dataframes ...
Creating ts tensor ...


array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         3.73856371e+04, 3.84319699e+04, 4.03453300e+04],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         3.91049000e+05, 4.54041000e+05, 5.52942000e+05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.80507000e+03, 2.13865000e+03, 2.68214000e+03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 0.00000000e

In [42]:
ts_loader = TimeSeriesLoader(ts_dataset= tourism_dataset,
                             model='nbeats',
                             offset = 0,
                             window_sampling_limit=50, 
                             input_size=8,
                             output_size=4,
                             idx_to_sample_freq=1,
                             batch_size=1024,
                             train_loader=True)

In [43]:
dataloader = iter(ts_loader)
batch = next(dataloader)
insample_y = batch['insample_y']
insample_x_t = batch['insample_x_t']
insample_mask = batch['insample_mask']
outsample_x_t = batch['outsample_x_t']
outsample_y = batch['outsample_y']
outsample_mask = batch['outsample_mask']

In [44]:
insample_y

array([[     0.,      0., 522300., ..., 551400., 556200., 602800.],
       [  6168.,   7198.,   8833., ...,  15460.,  20253.,  19508.],
       [     0.,      0.,      0., ..., 106644., 105738., 108639.],
       ...,
       [  1012.,    949.,    858., ...,   2086.,   2639.,   2515.],
       [ 15866.,  17340.,  19628., ...,  46490.,  66842.,  81357.],
       [ 21865.,  24574.,  27116., ...,  34435.,  36540.,  41579.]])

In [45]:
outsample_y

array([[615200., 713500., 752400., 763400.],
       [ 22882.,      0.,      0.,      0.],
       [109548., 121985., 104198., 100421.],
       ...,
       [  2913.,   2487.,   2495.,   2690.],
       [ 88909.,  68585.,  49149.,  61899.],
       [ 41118.,  44175.,      0.,      0.]])

In [47]:
outsample_mask

array([[1., 1., 1., 1.],
       [1., 0., 0., 0.],
       [1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 0., 0.]])

In [46]:
outsample_mask.mean()

0.94140625